## This notebook is part of the Apache Spark training delivered by CERN IT


Run this notebook from Jupyter with Python kernel
- When running on CERN SWAN, do not attach the notebook to a Spark cluster, but rather run it locally on the SWAN container (which is the default)
- If running this outside CERN SWAN, please make sure to have PySpark installed: `pip install pyspark`


In order to run this notebook as slides:
 - on SWAN click on the button "Enter/Exit RISE slideshow" in the ribbon
 - on other environments please make sure to have the RISE extension installed `pip install RISE`

### SPARK DataFrame Hands-On Lab
Contact: Luca.Canali@cern.ch

### Objective: Perform Basic DataFrame Operations
1. Creating DataFrames
2. Select columns
3. Add, rename and drop columns
4. Filtering rows
5. Aggregations

## Getting started: create the SparkSession

In [ ]:
!pip install pyspark


In [ ]:
# !pip install pyspark

from pyspark.sql import SparkSession

spark = (SparkSession.builder
          .master("local[*]") \
          .appName("DataFrame HandsOn 1") \
          .config("spark.ui.showConsoleProgress","false") \
          .getOrCreate()
        )

spark


The master `local[*]` means that the executors are in the same node that is running the driver. The `*` tells Spark to start as many executors as there are logical cores available

### Hands-On 1 - Construct a DataFrame from csv file
This demostrates how to read a csv file and construct a DataFrame.  
We will use the online retail dataset from Kaggle, credits: https://www.kaggle.com/datasets/vijayuv/onlineretail


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#### First, let's inspect the csv content

In [ ]:
#modify below code to use the downloaded dataset

!gzip -cd /content/online-retail-dataset.csv.gz 2>&1| head -n3


gzip: /content/online-retail-dataset.csv.gz: No such file or directory


In [4]:
online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"



In [ ]:
df = (spark.read
        .option("header", "true")
        .option("timestampFormat", "M/d/yyyy H:m")
        .csv("/content/drive/MyDrive/OnlineRetail.csv",
             schema=online_retail_schema)
     )


#### Inspect the data

In [ ]:
df.show(2, False)


+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerId|Country       |
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |2010-12-01 08:26:00|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN               |6       |2010-12-01 08:26:00|3.39     |17850     |United Kingdom|
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



#### Show columns

In [ ]:
df.printSchema()


root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Country: string (nullable = true)



### Hands-On 2 - Spark Transformations - select, add, rename and drop columns

Select dataframe columns

In [ ]:
# select single column

df.select("Country").show(2)


+--------------+
|       Country|
+--------------+
|United Kingdom|
|United Kingdom|
+--------------+
only showing top 2 rows



Select multiple columns


In [ ]:
df.select("StockCode","Description","UnitPrice").show(n=2, truncate=False)


+---------+----------------------------------+---------+
|StockCode|Description                       |UnitPrice|
+---------+----------------------------------+---------+
|85123A   |WHITE HANGING HEART T-LIGHT HOLDER|2.55     |
|71053    |WHITE METAL LANTERN               |3.39     |
+---------+----------------------------------+---------+
only showing top 2 rows



In [ ]:
df.columns


['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerId',
 'Country']

In [ ]:
# select first 5 columns
df.select(df.columns[0:5]).show(2)


+---------+---------+--------------------+--------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|
+---------+---------+--------------------+--------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|
+---------+---------+--------------------+--------+-------------------+
only showing top 2 rows



In [ ]:
# selects all the original columns and adds a new column that specifies high value item
(df.selectExpr(
   "*", # all original columns
   "(UnitPrice > 100) as HighValueItem")
   .show(2)
)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|HighValueItem|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|        false|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|        false|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
only showing top 2 rows



In [ ]:
# selects all the original columns and adds a new column that specifies high value item
(df.selectExpr(
  "sum(Quantity) as TotalQuantity",
  "cast(sum(UnitPrice) as int) as InventoryValue")
  .show()
)


+-------------+--------------+
|TotalQuantity|InventoryValue|
+-------------+--------------+
|      5176450|       2498803|
+-------------+--------------+



#### Adding, renaming and dropping columns

In [ ]:
# add a new column called InvoiceValue
from pyspark.sql.functions import expr
df_1 = (df
        .withColumn("InvoiceValue", expr("UnitPrice * Quantity"))
        .select("InvoiceNo","Description","UnitPrice","Quantity","InvoiceValue")
       )
df_1.show(2, False)

# rename InvoiceValue to LineTotal
df_2 = df_1.withColumnRenamed("InvoiceValue","LineTotal")
df_2.show(2, False)

# drop a column
df_2.drop("LineTotal").show(2, False)


+---------+----------------------------------+---------+--------+------------+
|InvoiceNo|Description                       |UnitPrice|Quantity|InvoiceValue|
+---------+----------------------------------+---------+--------+------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|2.55     |6       |15.299999   |
|536365   |WHITE METAL LANTERN               |3.39     |6       |20.34       |
+---------+----------------------------------+---------+--------+------------+
only showing top 2 rows

+---------+----------------------------------+---------+--------+---------+
|InvoiceNo|Description                       |UnitPrice|Quantity|LineTotal|
+---------+----------------------------------+---------+--------+---------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|2.55     |6       |15.299999|
|536365   |WHITE METAL LANTERN               |3.39     |6       |20.34    |
+---------+----------------------------------+---------+--------+---------+
only showing top 2 rows

+---------+---------

### Hands-On 3 - Spark Transformations - filter, sort and cast

In [ ]:
from pyspark.sql.functions import col

# select invoice lines with quantity > 50 and unitprice > 20
df.where(col("Quantity") > 20).where(col("UnitPrice") > 50).show(2)
df.filter(df.Quantity > 20).filter(df.UnitPrice > 50).show(2)
df.filter("Quantity > 20 and UnitPrice > 50").show(2)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   556444|    22502|PICNIC BASKET WIC...|      60|2011-06-10 15:28:00|    649.5|     15098|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   556444|    22502|PICNIC BASKET WIC...|      60|2011-06-10 15:28:00|    649.5|     15098|United Kingdom|
+---------+---------+------

In [ ]:
# select invoice lines with quantity > 100 or unitprice > 20
df.where((col("Quantity") > 100) | (col("UnitPrice") > 20)).show(2)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536378|    21212|PACK OF 72 RETROS...|     120|2010-12-01 09:37:00|     0.42|     14688|United Kingdom|
|     NULL|        D|            Discount|      -1|2010-12-01 09:41:00|     27.5|     14527|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [ ]:
from pyspark.sql.functions import desc, asc

# sort in the default order: ascending
df.orderBy(expr("UnitPrice")).show(2)

df.orderBy(col("Quantity").desc(), col("UnitPrice").asc()).show(10)


+---------+---------+---------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|    Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+---------------+--------+-------------------+---------+----------+--------------+
|     NULL|        B|Adjust bad debt|       1|2011-08-12 14:51:00|-11062.06|      NULL|United Kingdom|
|     NULL|        B|Adjust bad debt|       1|2011-08-12 14:52:00|-11062.06|      NULL|United Kingdom|
+---------+---------+---------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   581483|    23843|PAPER CRAFT 

### Hands-On 4 - Spark Transformations - aggregations
full list of built int functions - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions

In [ ]:
%%time
# Count distinct customers
from pyspark.sql.functions import countDistinct
df.select(countDistinct("CustomerID")).show()


+--------------------------+
|count(DISTINCT CustomerID)|
+--------------------------+
|                      4372|
+--------------------------+

CPU times: user 8.39 ms, sys: 5.31 ms, total: 13.7 ms
Wall time: 2 s


In [ ]:
%%time
# approx. distinct stock items
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("CustomerID", 0.1)).show()


+---------------------------------+
|approx_count_distinct(CustomerID)|
+---------------------------------+
|                             4336|
+---------------------------------+

CPU times: user 11.5 ms, sys: 2.14 ms, total: 13.6 ms
Wall time: 1.61 s


In [ ]:
# average, maximum and minimum purchase quantity
from pyspark.sql.functions import avg, max, min
( df.select(
    avg("Quantity").alias("avg_purchases"),
    max("Quantity").alias("max_purchases"),
    min("Quantity").alias("min_purchases"))
   .show()
)


+----------------+-------------+-------------+
|   avg_purchases|max_purchases|min_purchases|
+----------------+-------------+-------------+
|9.55224954743324|        80995|       -80995|
+----------------+-------------+-------------+



### Hands-On 5 - Spark Transformations - grouping and windows

In [ ]:
# count of items on the invoice
df.groupBy("InvoiceNo", "CustomerId").count().show(5)

# grouping with expressions
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show(5)


+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536573|     17025|    4|
|   537228|     17677|    1|
|   537419|     13495|   14|
|   538093|     12682|   33|
|   538648|     17937|    5|
+---------+----------+-----+
only showing top 5 rows

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536532| 25.36986301369863|  16.850272831671976|
|   537632|               1.0|                 0.0|
|   538708| 10.61111111111111|   7.150282736359209|
|   538877|14.258278145695364|   27.56989037543246|
|   538993| 9.333333333333334|   2.748737083745107|
+---------+------------------+--------------------+
only showing top 5 rows



### Read the csv file into DataFrame

`%%time` is an iPython magic https://ipython.readthedocs.io/en/stable/interactive/magics.html


It's possible to read files without specifying the schema. Some file formats (Parquet is one of them) include the schema, which means that Spark can start reading the file. For format without schema (csv, json...) Spark can infer the schema. Let's see what's the difference in terms of time and of results:

In [ ]:
online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"


In [ ]:
%%time
df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("/content/drive/MyDrive/OnlineRetail.csv",
             schema=online_retail_schema)


CPU times: user 2.57 ms, sys: 985 µs, total: 3.55 ms
Wall time: 35.1 ms


In [ ]:
%%time
df_infer = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("/content/drive/MyDrive/OnlineRetail.csv")


CPU times: user 12.2 ms, sys: 2.45 ms, total: 14.6 ms
Wall time: 2.72 s


## Exercises

Reminder: documentation at
https://spark.apache.org/docs/latest/api/python/index.html

If you didn't run the previous cells, run the following one:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("DataFrame HandsOn 1") \
        .config("spark.ui.showConsoleProgress","false") \
        .getOrCreate()

online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("/content/drive/MyDrive/OnlineRetail.csv",
             schema=online_retail_schema)


Task: Show 5 lines of the "description" column

In [ ]:
df.select("Description").show(5, truncate=False)


+-----------------------------------+
|Description                        |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



Task: Count the number of distinct invoices in the dataframe

In [ ]:
distinct_invoice_count = df.select("InvoiceNo").distinct().count()
print(f"Number of distinct invoices: {distinct_invoice_count}")


Number of distinct invoices: 22062


Task: Find out in which month most invoices have been issued

In [ ]:
from pyspark.sql.functions import month, col


monthly_invoices = df.groupBy(month("InvoiceDate").alias("Month")) \
                    .agg({"InvoiceNo": "count"}) \
                    .withColumnRenamed("count(InvoiceNo)", "InvoiceCount")
max_month = monthly_invoices.orderBy(col("InvoiceCount").desc()) \
                           .limit(1)
max_month.show()


+-----+------------+
|Month|InvoiceCount|
+-----+------------+
|   11|       83635|
+-----+------------+



Task: Filter the lines where the Quantity is more than 30

In [ ]:
filtered_df = df.filter(df.Quantity > 30)
filtered_df.show(5, truncate=False)
row_count = filtered_df.count()
print(f"Number of rows with Quantity > 30: {row_count}")


+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerId|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT  |32      |2010-12-01 08:34:00|1.69     |13047     |United Kingdom|
|536370   |10002    |INFLATABLE POLITICAL GLOBE     |48      |2010-12-01 08:45:00|0.85     |12583     |France        |
|536370   |22492    |MINI PAINT SET VINTAGE         |36      |2010-12-01 08:45:00|0.65     |12583     |France        |
|536371   |22086    |PAPER CHAIN KIT 50'S CHRISTMAS |80      |2010-12-01 09:00:00|2.55     |13748     |United Kingdom|
|536374   |21258    |VICTORIAN SEWING BOX LARGE     |32      |2010-12-01 09:09:00|10.95    |15100     |United Kingdom|
+---------+---------+---------------------------

Task: Show the four most sold items (by quantity)

In [ ]:
from pyspark.sql.functions import sum

sold_items = df.groupBy("Description").agg(sum("Quantity").alias("TotalQuantity"))
top_four_items = sold_items.orderBy(desc("TotalQuantity")).limit(4)
top_four_items.show(truncate=False)


+---------------------------------+-------------+
|Description                      |TotalQuantity|
+---------------------------------+-------------+
|WORLD WAR 2 GLIDERS ASSTD DESIGNS|53847        |
|JUMBO BAG RED RETROSPOT          |47363        |
|ASSORTED COLOUR BIRD ORNAMENT    |36381        |
|POPCORN HOLDER                   |36334        |
+---------------------------------+-------------+



Bonus question: why do these two operations return different results? Hint: look at the documentation

In [ ]:
print(df.select("InvoiceNo").distinct().count())
from pyspark.sql.functions import countDistinct
df.select(countDistinct("InvoiceNo")).show()


22062
+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    22061|
+-------------------------+



The difference between df.select("InvoiceNo").distinct().count() and df.select(countDistinct("InvoiceNo")).show() in PySpark arises from how they handle null values, as hinted by the documentation. The first operation, distinct().count(), selects the "InvoiceNo" column, removes duplicate rows (treating null as a distinct value), and counts all resulting rows, including any with null, because count() simply tallies rows without regard to nullness—e.g., for values [536365, 536366, null, 536365, null], it yields 3 (536365, 536366, null). The second operation, using countDistinct("InvoiceNo"), is an aggregation function that counts only distinct non-null values in the column, explicitly excluding null as per its documentation, so the same data yields 2 (536365, 536366); it then displays this single value in a DataFrame via show(). In the Online Retail dataset, if "InvoiceNo" contains any null entries, the first method will return a higher count by including them, while the second ignores them, making countDistinct more suitable for counting unique non-null invoices, whereas distinct().count() reflects all unique values, nulls included.